In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
import numpy as np
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.optim import lr_scheduler

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
#print the pytorch version
# device = torch.cuda.is_available()
device ="cuda" if torch.cuda.is_available() else "cpu"
device, torch.__version__

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    
print(train_on_gpu)

CUDA is available!  Training on GPU ...
True


In [3]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    inp = inp.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    inp = np.clip(inp, 0, 1)
    
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [5]:
model = models.resnet34(pretrained=True)

In [6]:
num_inp = model.fc.in_features
model.fc = nn.Linear(num_inp, 5)
if train_on_gpu:
    model.cuda()

In [7]:
criterion = torch.nn.CrossEntropyLoss()
#only train the classifier part, the features part are frozen
optimizer = optim.SGD(model.fc.parameters(), lr=0.015, momentum=0.9)

In [8]:
# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)

In [9]:
#TO UNFREEZE THE WEIGHTS FOR THE SECOND TIME TRAINING AND DON'T FORGET TO CHANGE THE OPTIMIZER TOO!
for param in model.parameters():
    param.requires_grad = True
    
# train the WHOLE part instead of just the classifier!, THIS IS THE SECOND ITERATION OF THE TRAINING AND REDUCE THE LEARNING RATE TOO!
optimizer = optim.SGD(model.parameters(), lr=0.0002, momentum=0.9)

In [10]:
model.load_state_dict(torch.load('model_cifar_resnet152.pt'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
data_dir = './dataset'
test_dir = data_dir + '/valid'

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data))

In [12]:
dataiter = iter(test_loader)
images, labels =dataiter.next()
images, labels = images.to(device), labels.to(device)

In [13]:
classes = ('0', '1', '2', '3', '4')

In [14]:
imshow(torchvision.utils.make_grid(images))
#print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(250)))

TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [15]:
outputs = model(images)
_, pred = torch.max(outputs, 1)
#print('Predicted: ', ' '.join('%5s' % classes[pred[j]] for j in range(250)))

In [16]:
for j in range(250):
    print(classes[pred[j]])

0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
2
2
2
2
2
2
2
2
2
2
4
4
4
4
4
4
4
4
4
4
4
4
4
0
0
0
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
4
4
4
4
4
4
4
4
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [17]:
allFiles, _ = map(list, zip(*test_loader.dataset.samples))
for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    outputs = model(images)
    _, pred = torch.max(outputs.data, 1)
    for j in range(inputs.size()[0]):
        print('"'+allFiles[ i * len(test_data) + j ] + '": ' + classes[pred[j]]+',')

"./dataset/valid/valid/10_13_20211119093410011.jpg": 0,
"./dataset/valid/valid/10_13_20211119093410088.jpg": 0,
"./dataset/valid/valid/10_13_20211119093410165.jpg": 0,
"./dataset/valid/valid/10_13_20211119161549009.jpg": 1,
"./dataset/valid/valid/10_13_20211119161549042.jpg": 1,
"./dataset/valid/valid/10_13_20211119161549072.jpg": 1,
"./dataset/valid/valid/10_13_20211119161549093.jpg": 1,
"./dataset/valid/valid/10_13_20211119161549114.jpg": 1,
"./dataset/valid/valid/10_13_20211119161549135.jpg": 1,
"./dataset/valid/valid/10_13_20211119161549162.jpg": 1,
"./dataset/valid/valid/10_13_20211119161549183.jpg": 1,
"./dataset/valid/valid/10_13_20211119161549213.jpg": 1,
"./dataset/valid/valid/10_13_20211119161549237.jpg": 1,
"./dataset/valid/valid/10_13_20211122043639009.jpg": 0,
"./dataset/valid/valid/10_13_20211122043639108.jpg": 0,
"./dataset/valid/valid/10_13_20211122043639196.jpg": 0,
"./dataset/valid/valid/10_13_20211123044420030.jpg": 0,
"./dataset/valid/valid/10_13_20211123044420107.j